In [161]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels as sm


In [ ]:
imp

In [162]:
df = pd.read_csv('./data/stock-market-data.csv')


In [163]:
col = df.columns


In [164]:
df[:4]


,symbol,date,pre_close,open,high,low,close,volume,amount,adj_factor,capt,index_w50,index_w300,index_w500,industry
0,600000.SH,20120104,8.49,8.54,8.56,8.39,8.41,34201379,290229551.00,6.66,125500555680.00,0.05,0.02,0.00,BANKS
1,600000.SH,20120105,8.41,8.47,8.82,8.47,8.65,132116203,1144753023.00,6.66,129082022192.00,0.05,0.02,0.00,BANKS
2,600000.SH,20120106,8.65,8.63,8.78,8.62,8.71,61778687,537043761.00,6.66,129977388820.00,0.05,0.02,0.00,BANKS
3,600000.SH,20120109,8.71,8.72,8.99,8.68,8.95,80136249,711429611.00,6.66,133558855331.00,0.05,0.02,0.00,BANKS


In [165]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276900 entries, 0 to 276899
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   symbol      276900 non-null  object 
 1   date        276900 non-null  int64  
 2   pre_close   276900 non-null  float64
 3   open        276900 non-null  float64
 4   high        276900 non-null  float64
 5   low         276900 non-null  float64
 6   close       276900 non-null  float64
 7   volume      276900 non-null  int64  
 8   amount      276900 non-null  float64
 9   adj_factor  276900 non-null  float64
 10  capt        276900 non-null  float64
 11  index_w50   276900 non-null  float64
 12  index_w300  276900 non-null  float64
 13  index_w500  276900 non-null  float64
 14  industry    276900 non-null  object 
dtypes: float64(11), int64(2), object(2)
memory usage: 31.7+ MB


In [166]:
df[['symbol', 'industry']] = df[['symbol', 'industry']].apply(
    lambda x: x.astype("string"))


In [167]:
# 1
a1 = set(df['symbol'][df['symbol'].apply(lambda x: x.count('8') > 0)])


In [168]:
# 2
# df['Flag'] = np.where(df['close'] > df['pre_close'],'Increase',np.where(df['close'] < df['pre_close'],'Decrease','Flat'))

conditions = [
    df['close'] > df['pre_close'],
    df['close'] < df['pre_close']
]

results = ['Increase', 'Decrease']
df['Flag'] = np.select(conditions, results, default='Flat')

df.groupby(['date', 'Flag']).agg({'symbol': 'nunique'})


symbol
date     Flag            
20120104 Decrease    2007
         Flat         122
         Increase     191
20120105 Decrease    2071
         Flat         117
...                   ...
20120628 Flat         176
         Increase     427
20120629 Decrease     358
         Flat         174
         Increase    1890

[351 rows x 1 columns]

In [239]:
dff = pd.DataFrame({"a": [1, 2, 3, 4, 5], "b": [10, 11, 12, 13, 14]})

In [240]:
dff.head()

,a,b
0,1,10
1,2,11
2,3,12
3,4,13
4,5,14


In [264]:
dff['a'].rolling(2,min_periods=2).apply(lambda x: 1 if (x.iloc[0]>1) & (x.iloc[1]>1) else 0)

0    NaN
1   0.00
2   1.00
3   1.00
4   1.00
Name: a, dtype: float64

In [266]:
dff['a'].iloc[0]

1

In [169]:
df['exchange'] = df['symbol'].str[-2:]
df.head()


,symbol,date,pre_close,open,high,low,close,volume,amount,adj_factor,capt,index_w50,index_w300,index_w500,industry,Flag,exchange
0,600000.SH,20120104,8.49,8.54,8.56,8.39,8.41,34201379,290229551.00,6.66,125500555680.00,0.05,0.02,0.00,BANKS,Decrease,SH
1,600000.SH,20120105,8.41,8.47,8.82,8.47,8.65,132116203,1144753023.00,6.66,129082022192.00,0.05,0.02,0.00,BANKS,Increase,SH
2,600000.SH,20120106,8.65,8.63,8.78,8.62,8.71,61778687,537043761.00,6.66,129977388820.00,0.05,0.02,0.00,BANKS,Increase,SH
3,600000.SH,20120109,8.71,8.72,8.99,8.68,8.95,80136249,711429611.00,6.66,133558855331.00,0.05,0.02,0.00,BANKS,Increase,SH
4,600000.SH,20120110,8.95,8.95,9.10,8.88,9.07,72004632,647206633.00,6.66,135349588587.00,0.05,0.02,0.00,BANKS,Increase,SH


In [170]:
# 3
df.groupby(['date', 'exchange', 'Flag']).agg(
    {'symbol': 'nunique'}).reset_index()[:5]


,date,exchange,Flag,symbol
0,20120104,SH,Decrease,794
1,20120104,SH,Flat,42
2,20120104,SH,Increase,85
3,20120104,SZ,Decrease,1213
4,20120104,SZ,Flat,80


In [171]:
# 4

df.query("index_w300 > 0").groupby(['date', 'Flag']).agg(
    {'symbol': 'nunique'}).reset_index()[:5]


,date,Flag,symbol
0,20120104,Decrease,275
1,20120104,Flat,5
2,20120104,Increase,20
3,20120105,Decrease,242
4,20120105,Flat,8


In [172]:
# 5
df.groupby(['date', 'industry']).agg({'symbol': 'nunique'}).reset_index()[:5]


,date,industry,symbol
0,20120104,AERODEF,10
1,20120104,AIRLINE,12
2,20120104,AUTO,85
3,20120104,BANKS,16
4,20120104,BEV,30


In [173]:
# 6
df1 = df.groupby(['date', 'industry']).agg(
    {'symbol': 'nunique', 'amount': 'sum'}).reset_index()


In [174]:
df1[['max_n', 'max_a']] = df1.groupby(
    ['date'])[['symbol', 'amount']].transform('max')


In [175]:
df1.query('symbol == max_n & amount == max_a')


,date,industry,symbol,amount,max_n,max_a
15,20120104,HDWRSEMI,224,6878138136.80,224,6878138136.80
79,20120106,HDWRSEMI,224,5979258564.90,224,5979258564.90
751,20120213,HDWRSEMI,227,14602692451.80,227,14602692451.80
847,20120216,HDWRSEMI,227,15064632475.20,227,15064632475.20
879,20120217,HDWRSEMI,229,11451347414.00,229,11451347414.00
911,20120220,HDWRSEMI,230,15251810573.60,230,15251810573.60
943,20120221,HDWRSEMI,230,15491841616.60,230,15491841616.60
975,20120222,HDWRSEMI,230,19900505866.60,230,19900505866.60
1167,20120301,HDWRSEMI,231,14865249097.90,231,14865249097.90
1199,20120302,HDWRSEMI,231,16208547305.30,231,16208547305.30


In [176]:
del df1


In [177]:
# 7

conditions = [
    (df['close'] - df['pre_close']) / df['pre_close'] > 0.05,
    (df['close'] - df['pre_close']) / df['pre_close'] < - 0.05
]

results = ['Increase 5%', 'Decrease 5%']
df['Flag%'] = np.select(conditions, results, default='Flat')

df.query('`Flag%` != "Flat"').groupby(['date', 'Flag%']).agg(
    {'symbol': 'nunique'}).reset_index()[:5]


,date,Flag%,symbol
0,20120104,Decrease 5%,277
1,20120104,Increase 5%,17
2,20120105,Decrease 5%,885
3,20120105,Increase 5%,10
4,20120106,Decrease 5%,66


In [178]:
# 8
df['rate%'] = (df['close'] - df['pre_close']) / df['pre_close']

df.sort_values(by='rate%', ascending=False, inplace=True)


In [179]:

df1 = df.groupby(['date']).agg(top10=('amount', lambda x: sum(x[:10])),
                               last10=('amount', lambda x: sum(x[-10:])))


In [180]:
pd.options.display.float_format = '{:.2f}'.format


In [181]:
pd.set_option("display.max_rows", 150)


In [182]:
df1['top/bottom'] = df1['top10']/df1['last10']
df1


,top10,last10,top/bottom
date,,,
20120104,660522056.10,456926638.10,1.45
20120105,660035624.90,383022701.40,1.72
20120106,947170129.80,703279193.90,1.35
20120109,1586198245.50,558059839.50,2.84
20120110,1658431604.10,2948836472.30,0.56
20120111,3024195777.40,2144084175.70,1.41
20120112,1215494588.20,2950479801.60,0.41
20120113,2611351394.70,875104112.30,2.98
20120116,458861322.50,625366458.30,0.73


In [183]:
# 9
df['ret_open'] = df['open']/df['pre_close'] > 1.015
df['ret_close'] = df['close']/df['pre_close'] > 1.015


In [184]:
df.groupby('date')[['ret_open', 'ret_close']].sum().reset_index()


,date,ret_open,ret_close
0,20120104,325,70
1,20120105,27,60
2,20120106,56,743
3,20120109,73,2142
4,20120110,95,2125
5,20120111,75,354
6,20120112,39,258
7,20120113,46,56
8,20120116,19,40
9,20120117,120,2180


In [160]:
# 10
flag_open = []

for i in range(3, df.shape[0]-1):

    flag_open.append(
        sum(map(int, [df['ret_open'][i-3], df['ret_open'][i-2], df['ret_open'][i-1]])))


In [ ]:
filled_open = [None] * 4 + flag_open


In [ ]:
df['flag_open'] = pd.Series(filled_open)


In [ ]:
df.groupby('date')[['flag_open']].sum()


In [192]:
ids=[k for k in df.columns if 'index' not in k]

In [196]:
df2=df.melt(id_vars=ids,var_name='Index',value_name='index%')


In [224]:
df2.query('`index%`>0').groupby(['Index','date','exchange']).agg(stkcd_amount = ('symbol','nunique')).reset_index().sort_values(['Index'], key = lambda x: x.str.len(),ascending = True)



,Index,date,exchange,stkcd_amount
292,index_w50,20120405,SH,50
263,index_w50,20120221,SH,50
264,index_w50,20120222,SH,50
265,index_w50,20120223,SH,50
266,index_w50,20120224,SH,50
...,...,...,...,...
150,index_w300,20120502,SH,205
149,index_w300,20120427,SZ,95
148,index_w300,20120427,SH,205
160,index_w300,20120509,SH,205


In [226]:
df2.query('`index%`>0').groupby(['Index','date','industry']).agg(stkcd_amount = ('symbol','nunique')).reset_index().sort_values(['Index'], key = lambda x: x.str.len(),ascending = True)



,Index,date,industry,stkcd_amount
4562,index_w50,20120410,TRDDIST,1
4183,index_w50,20120306,HDWRSEMI,1
4184,index_w50,20120306,MACH,4
4185,index_w50,20120306,MTLMIN,10
4186,index_w50,20120306,RDRLTRAN,1
...,...,...,...,...
2374,index_w300,20120508,BEV,10
2373,index_w300,20120508,BANKS,16
2372,index_w300,20120508,AUTO,12
2383,index_w300,20120508,FOODPROD,9


In [236]:
df2.query('`index%`>0 & Index == "index_w300"').sort_values(['date','index%'],ascending=[True,False]).groupby('date').head(10)[['date','symbol']].reset_index()



,index,date,symbol
0,499360,20120104,600036.SH
1,452665,20120104,600016.SH
2,495697,20120104,601318.SH
3,460015,20120104,601328.SH
4,435416,20120104,601166.SH
...,...,...,...
1165,310288,20120629,601166.SH
1166,341007,20120629,600000.SH
1167,287737,20120629,600030.SH
1168,376849,20120629,000002.SZ


In [227]:
df2.head()


,symbol,date,pre_close,open,high,low,close,volume,amount,adj_factor,capt,industry,Flag,exchange,Flag%,rate%,ret_open,ret_close,Index,index%
0,600401.SH,20120217,2.78,12.50,13.50,10.61,10.74,61977060,703751398.00,5.23,1941504340.00,MATERIAL,Increase,SH,Increase 5%,2.86,True,True,index_w50,0.00
1,603002.SH,20120518,3.60,8.01,8.77,8.01,8.56,68880207,573951279.00,1.00,684800000.00,CHEM,Increase,SH,Increase 5%,1.38,True,True,index_w50,0.00
2,300297.SZ,20120315,16.00,28.26,31.00,28.21,30.98,13002333,382154777.20,1.00,610306000.00,SOFTWARE,Increase,SZ,Increase 5%,0.94,True,True,index_w50,0.00
3,601929.SH,20120223,7.00,10.88,14.16,10.88,13.12,210791050,2401265522.00,1.00,2938880000.00,MEDIA,Increase,SH,Increase 5%,0.87,True,True,index_w50,0.00
4,300288.SZ,20120216,22.44,38.10,42.37,38.10,40.96,10191696,408126414.80,1.00,439091200.00,SOFTWARE,Increase,SZ,Increase 5%,0.83,True,True,index_w50,0.00


In [267]:
df2.head()

,symbol,date,pre_close,open,high,low,close,volume,amount,adj_factor,capt,industry,Flag,exchange,Flag%,rate%,ret_open,ret_close,Index,index%
0,600401.SH,20120217,2.78,12.50,13.50,10.61,10.74,61977060,703751398.00,5.23,1941504340.00,MATERIAL,Increase,SH,Increase 5%,2.86,True,True,index_w50,0.00
1,603002.SH,20120518,3.60,8.01,8.77,8.01,8.56,68880207,573951279.00,1.00,684800000.00,CHEM,Increase,SH,Increase 5%,1.38,True,True,index_w50,0.00
2,300297.SZ,20120315,16.00,28.26,31.00,28.21,30.98,13002333,382154777.20,1.00,610306000.00,SOFTWARE,Increase,SZ,Increase 5%,0.94,True,True,index_w50,0.00
3,601929.SH,20120223,7.00,10.88,14.16,10.88,13.12,210791050,2401265522.00,1.00,2938880000.00,MEDIA,Increase,SH,Increase 5%,0.87,True,True,index_w50,0.00
4,300288.SZ,20120216,22.44,38.10,42.37,38.10,40.96,10191696,408126414.80,1.00,439091200.00,SOFTWARE,Increase,SZ,Increase 5%,0.83,True,True,index_w50,0.00


In [271]:
list(df2.loc[:,'symbol':'ret_close'].columns)

['symbol',
 'date',
 'pre_close',
 'open',
 'high',
 'low',
 'close',
 'volume',
 'amount',
 'adj_factor',
 'capt',
 'industry',
 'Flag',
 'exchange',
 'Flag%',
 'rate%',
 'ret_open',
 'ret_close']

In [273]:
df2.pivot(index = list(df2.loc[:,'symbol':'ret_close'].columns),columns='Index').reset_index().head()

symbol      date pre_close  open  high   low close    volume  \
Index                                                                    
0      000001.SZ  20120104     15.59 15.59 15.59 15.15 15.16  14791098   
1      000001.SZ  20120105     15.16 15.16 15.55 15.13 15.39  24408005   
2      000001.SZ  20120106     15.39 15.35 15.51 15.15 15.35  13315115   
3      000001.SZ  20120109     15.35 15.36 15.81 15.25 15.78  22113866   
4      000001.SZ  20120110     15.78 15.78 15.78 15.78 15.78         0   

            amount adj_factor  ... industry      Flag exchange Flag% rate%  \
Index                          ...                                           
0     227563714.50      35.91  ...    BANKS  Decrease       SZ  Flat -0.03   
1     375880600.60      35.91  ...    BANKS  Increase       SZ  Flat  0.02   
2     204143668.30      35.91  ...    BANKS  Decrease       SZ  Flat -0.00   
3     345938687.20      35.91  ...    BANKS  Increase       SZ  Flat  0.03   
4             0.00      35.91  ...    BANKS      Flat       SZ  Flat  0.00   

      ret_open ret_close     index%                       
Index                    index_w300 index_w50 index_w500  
0        False     False       0.01      0.00       0.00  
1        False      True       0.01      0.00       0.00  
2        False     False       0.01      0.00       0.00  
3        False      True       0.01      0.00       0.00  
4        False     False       0.01      0.00       0.00  

[5 rows x 21 columns]

In [282]:
df.loc[df.groupby(['date','industry'])['amount'].idxmax()][['date','industry','symbol']]

,date,industry,symbol
144101,20120104,AERODEF,000768.SZ
2457,20120104,AIRLINE,600029.SH
9628,20120104,AUTO,600104.SH
3042,20120104,BANKS,600036.SH
49563,20120104,BEV,600519.SH
...,...,...,...
3626,20120629,REALEST,600048.SH
72845,20120629,RETAIL,600739.SH
253396,20120629,SOFTWARE,300104.SZ
36341,20120629,TRDDIST,600366.SH


In [286]:
#25
a=df.groupby(['date','industry']).agg(max_a =('amount','max'),min_a = ('amount','min')).reset_index()
a['ratio']=a['max_a']/a['min_a']
a.head()

,date,industry,max_a,min_a,ratio
0,20120104,AERODEF,172921879.70,13489241.70,12.82
1,20120104,AIRLINE,112322418.00,2370467.80,47.38
2,20120104,AUTO,388571668.00,0.00,inf
3,20120104,BANKS,620703189.00,53406484.00,11.62
4,20120104,BEV,637232812.00,0.00,inf


In [291]:
#26
a=df.groupby(['date','industry']).agg({'amount':lambda x : sum(x.nlargest(5))})

In [297]:
#29
df['avg_amount'] = df.groupby(['date'])['amount'].transform('mean')
df['avg_industry_amount'] = df.groupby(['date','industry'])['amount'].transform('mean')
df.query('avg_industry_amount > avg_amount')[['date','industry']].drop_duplicates().sort_values(['date','industry'],ascending=True)


,date,industry
144101,20120104,AERODEF
105025,20120104,BANKS
130295,20120104,BEV
207515,20120104,CONMAT
80919,20120104,DVFININS
...,...,...
60677,20120629,INDCONG
89810,20120629,MARINE
106982,20120629,MEDIA
103194,20120629,MTLMIN


In [298]:
df.sort_values(['symbol','date'],ascending=True,inplace=True)

In [299]:
df['flag_up_down'] = np.select([df['pre_close']>df['close'],df['pre_close']<df['close']],['down','up'],default='flat')

In [ ]:
df.groupby('symbol')['']

In [301]:
pd.set_option('display.expand_frame_repr', False)

In [305]:
pd.set_option('display.max_columns', None)

In [ ]:
pd.set_option('display.expand_frame_repr', False)

In [306]:
df.head()

,symbol,date,pre_close,open,high,low,close,volume,amount,adj_factor,capt,index_w50,index_w300,index_w500,industry,Flag,exchange,Flag%,rate%,ret_open,ret_close,avg_amount,avg_industry_amount,flag_up_down
108416,000001.SZ,20120104,15.59,15.59,15.59,15.15,15.16,14791098,227563714.50,35.91,47077237468.00,0.00,0.01,0.00,BANKS,Decrease,SZ,Flat,-0.03,False,False,34277999.13,225750794.69,down
108417,000001.SZ,20120105,15.16,15.16,15.55,15.13,15.39,24408005,375880600.60,35.91,47791469962.00,0.00,0.01,0.00,BANKS,Increase,SZ,Flat,0.02,False,True,37897894.59,494676233.74,up
108418,000001.SZ,20120106,15.39,15.35,15.51,15.15,15.35,13315115,204143668.30,35.91,47667255615.00,0.00,0.01,0.00,BANKS,Decrease,SZ,Flat,-0.00,False,False,32480550.72,288272343.82,down
108419,000001.SZ,20120109,15.35,15.36,15.81,15.25,15.78,22113866,345938687.20,35.91,49002559844.00,0.00,0.01,0.00,BANKS,Increase,SZ,Flat,0.03,False,True,48770458.09,488458345.68,up
108420,000001.SZ,20120110,15.78,15.78,15.78,15.78,15.78,0,0.00,35.91,49002559844.00,0.00,0.01,0.00,BANKS,Flat,SZ,Flat,0.00,False,False,71647581.55,420013583.36,flat
